In [117]:
import pandas as pd

In [118]:
cluster4 = pd.read_csv('cluster4.csv', index_col=0)
#cluster4.drop(cluster4.loc[1], inplace=True)
cluster4.reset_index(drop = True, inplace=True)
cluster4.head(15)

,Cluster Label,Zip Code,City,Community,Estimated Median Income,Longitude,Latitude,Total Population,Median Age,Total Males,Total Females,Total Households,Average Household Size,Factor
0,4,90011,Los Angeles,Los Angeles (Southeast Los Angeles),40940,-118.25868,34.007063,103892,26.2,52794,51098,22168,4.67,4253338480
1,4,90019,Los Angeles,"Los Angeles (Arlington Heights, Country Club P...",56389,-118.34015,34.048411,64458,35.8,31442,33016,23344,2.70,3634722162
2,4,90022,Los Angeles,East Los Angeles,46051,-118.15581,34.023662,67179,29.8,33216,33963,17023,3.94,3093660129
3,4,90026,Los Angeles,"Los Angeles (Echo Park, Silver Lake)",65269,-118.26596,34.078510,67869,34.0,34515,33354,24956,2.68,4429741761
4,4,90044,Los Angeles,"Athens, Los Angeles (South Los Angeles)",35981,-118.29158,33.953814,89779,28.6,43128,46651,25144,3.55,3230338199
5,4,90201,Bell,"Bell, Bell Gardens, Cudahy",44046,-118.17385,33.972814,101279,27.8,50658,50621,24104,4.16,4460934834
6,4,90250,Hawthorne,Hawthorne (Holly Park),56304,-118.35092,33.914614,93193,31.9,45113,48080,31087,2.98,5247138672
7,4,90255,Huntington Park,"Huntington Park, Walnut Park",44962,-118.21773,33.978404,75066,29.1,37525,37541,18419,4.06,3375117492
8,4,90262,Lynwood,Lynwood,52313,-118.20186,33.924642,69745,27.8,33919,35826,14669,4.57,3648570185
9,4,90280,South Gate,South Gate,52321,-118.19490,33.944264,94396,29.4,46321,48075,23278,4.05,4938893116


In [119]:
import foursquare
import requests
CLIENT_ID = foursquare.CLIENT_ID
CLIENT_SECRET = foursquare.CLIENT_SECRET
VERSION = '20210501' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [120]:
record = cluster4.iloc[10]

In [160]:
import itertools
offset = 0
venues_list=[]

for _ in itertools.repeat(None, 4):
    url = 'https://api.foursquare' \
          '.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},' \
          '{}&radius={}&limit={}&offset={}&categoryId={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        record['Latitude'],
        record['Longitude'],
        4000,
        50,
        offset,
        '4d4b7105d754a06374d81259')

    offset += 50

    # make the GET request
   # try:
    try:
        #results = requests.get(url).json()["response"]['venues']
        results = requests.get(url).json()["response"]['groups'][0]['items']
        for v in results:
            try:
                city = v['venue']['location']['city']
            except:
                city = 'N. A.'

            try:
                postalCode = str(v['venue']['location']['postalCode'])
            except:
                postalCode = str(record['Zip Code'])

            #if postalCode == str(record['Zip Code']):
            venues_list.append((
                        record['Zip Code'],
                        record['Community'],
                        record['Latitude'],
                        record['Longitude'],
                        v['venue']['name'],
                        # v['venue']['location']['lat'],
                        # v['venue']['location']['lng'],
                        v['venue']['categories'][0]['name'],
                        city, postalCode))
    except:
        print('done!')
        break
   # except:
   #     print('done')


#nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues = pd.DataFrame(venues_list, columns=['Zip Code', 'Community',
              'Zip Code Latitude', 'Zip Code Longitude', 'Venue',
              # 'Venue Latitude',
              # 'Venue Longitude',
              'Venue Category',
              'City', 'PostalCode'])
nearby_venues.count()

Zip Code              141
Community             141
Zip Code Latitude     141
Zip Code Longitude    141
Venue                 141
Venue Category        141
City                  141
PostalCode            141
dtype: int64

In [161]:
import numpy as np
np.sum(nearby_venues['City'] == 'Norwalk')
#nearby_venues[nearby_venues['City']== 'Norwalk'].count()

74

In [162]:
nearby_venues.head()

,Zip Code,Community,Zip Code Latitude,Zip Code Longitude,Venue,Venue Category,City,PostalCode
0,90650,Norwalk,33.906914,-118.08339,In-N-Out Burger,Fast Food Restaurant,Norwalk,90650
1,90650,Norwalk,33.906914,-118.08339,Douglas Charbroiled Burgers,American Restaurant,Norwalk,90650
2,90650,Norwalk,33.906914,-118.08339,Noemi's Mexican Food,Mexican Restaurant,Norwalk,90650
3,90650,Norwalk,33.906914,-118.08339,Los Compadres Mexican Restaurant & Sports Bar,Mexican Restaurant,Norwalk,90650
4,90650,Norwalk,33.906914,-118.08339,Tacos Los Gueros,Mexican Restaurant,Norwalk,90650


In [163]:
nearby_venues.to_csv(path_or_buf='norwalk_test.csv')

In [168]:
#ppds = nearby_venues.drop(nearby_venues[nearby_venues['City'] !=
#                             'Norwalk'].index)

#ppds = nearby_venues[nearby_venues['PostalCode']=='9065']
ppds = nearby_venues[nearby_venues['PostalCode'].str.contains(r'90650')]
ppds.head(10)

,Zip Code,Community,Zip Code Latitude,Zip Code Longitude,Venue,Venue Category,City,PostalCode
0,90650,Norwalk,33.906914,-118.08339,In-N-Out Burger,Fast Food Restaurant,Norwalk,90650
1,90650,Norwalk,33.906914,-118.08339,Douglas Charbroiled Burgers,American Restaurant,Norwalk,90650
2,90650,Norwalk,33.906914,-118.08339,Noemi's Mexican Food,Mexican Restaurant,Norwalk,90650
3,90650,Norwalk,33.906914,-118.08339,Los Compadres Mexican Restaurant & Sports Bar,Mexican Restaurant,Norwalk,90650
4,90650,Norwalk,33.906914,-118.08339,Tacos Los Gueros,Mexican Restaurant,Norwalk,90650
5,90650,Norwalk,33.906914,-118.08339,Señor Baja,Mexican Restaurant,Norwalk,90650
6,90650,Norwalk,33.906914,-118.08339,Kung Pao Bowl,Chinese Restaurant,Norwalk,90650
7,90650,Norwalk,33.906914,-118.08339,The Habit Burger Grill,Burger Joint,Norwalk,90650
8,90650,Norwalk,33.906914,-118.08339,Ho Ho China,Chinese Restaurant,Norwalk,90650
9,90650,Norwalk,33.906914,-118.08339,Grill Town,Korean Restaurant,Norwalk,90650


In [169]:
ppds['City'].count()

82

In [170]:
#ppds.to_csv(path_or_buf='nearby_venues_3500.csv')
ppds.to_csv(path_or_buf='ppds.csv')

In [171]:
df = ppds[['Venue Category', 'Zip Code']]
df_grouped = df.groupby('Venue Category').agg(['mean', 'count'])
df_sorted = df_grouped.sort_values(by=[('Zip Code','count')], ascending=False)
#df_grouped.sort_values(by='count')
df_sorted

Zip Code      
                         mean count
Venue Category                     
Fast Food Restaurant    90650    14
Mexican Restaurant      90650    12
Sandwich Place          90650     8
Pizza Place             90650     7
Chinese Restaurant      90650     6
American Restaurant     90650     5
Donut Shop              90650     5
Asian Restaurant        90650     4
Food Truck              90650     4
Bakery                  90650     2
Burger Joint            90650     2
Thai Restaurant         90650     1
Taco Place              90650     1
Sushi Restaurant        90650     1
Steakhouse              90650     1
Food Court              90650     1
Korean Restaurant       90650     1
Fried Chicken Joint     90650     1
Food                    90650     1
Fish & Chips Shop       90650     1
Donburi Restaurant      90650     1
Buffet                  90650     1
Breakfast Spot          90650     1
Wings Joint             90650     1

In [172]:
ppds.head(20)

,Zip Code,Community,Zip Code Latitude,Zip Code Longitude,Venue,Venue Category,City,PostalCode
0,90650,Norwalk,33.906914,-118.08339,In-N-Out Burger,Fast Food Restaurant,Norwalk,90650
1,90650,Norwalk,33.906914,-118.08339,Douglas Charbroiled Burgers,American Restaurant,Norwalk,90650
2,90650,Norwalk,33.906914,-118.08339,Noemi's Mexican Food,Mexican Restaurant,Norwalk,90650
3,90650,Norwalk,33.906914,-118.08339,Los Compadres Mexican Restaurant & Sports Bar,Mexican Restaurant,Norwalk,90650
4,90650,Norwalk,33.906914,-118.08339,Tacos Los Gueros,Mexican Restaurant,Norwalk,90650
5,90650,Norwalk,33.906914,-118.08339,Señor Baja,Mexican Restaurant,Norwalk,90650
6,90650,Norwalk,33.906914,-118.08339,Kung Pao Bowl,Chinese Restaurant,Norwalk,90650
7,90650,Norwalk,33.906914,-118.08339,The Habit Burger Grill,Burger Joint,Norwalk,90650
8,90650,Norwalk,33.906914,-118.08339,Ho Ho China,Chinese Restaurant,Norwalk,90650
9,90650,Norwalk,33.906914,-118.08339,Grill Town,Korean Restaurant,Norwalk,90650


In [173]:
df = ppds[['Venue Category']]
df_count = df.groupby('Venue Category').size().to_frame('Count').reset_index()
df_sorted = df_count.sort_values(by=[('Count')], ascending=False)
df_sorted

,Venue Category,Count
9,Fast Food Restaurant,14
16,Mexican Restaurant,12
18,Sandwich Place,8
17,Pizza Place,7
6,Chinese Restaurant,6
0,American Restaurant,5
8,Donut Shop,5
1,Asian Restaurant,4
13,Food Truck,4
2,Bakery,2


In [174]:
import plotly.express as px
fig = px.bar(df_sorted, x="Venue Category", y="Count", title='Distribution of Food Venues in Norwalk')
fig.show()